In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

In [3]:
year = 2019
day = 14

14

In [4]:
input = get_input(year, day)

"2 WZMS, 3 NPNFD => 5 SLRGD\n4 QTFCJ, 1 RFZF => 1 QFQPN\n2 LCDPV => 6 DGPND\n1 MVSHM, 3 XSDR, 1 RSJD => 6 GNKB\n6 XJRML, 1 LCDPV => 7 HTSJ\n3 LQBX => 3 GKNTG\n2 NZMLP, 5 FTNZQ => 2 QSLTQ\n8 WZMS, 4 XSDR, 2 NPNFD => 9 CJVT\n16 HFHB, 1 TRVQG => 8 QTBQ\n177 ORE => 7 DNWGS\n10 ZJFM, 4" ⋯ 1249 bytes ⋯ "X\n170 ORE => 9 GDBNV\n5 RSJD, 3 CZHM, 1 GNKB => 6 HNJW\n14 HTSJ => 7 FXJQX\n11 NPNFD, 1 LCDPV, 2 FXJQX => 6 RSJD\n9 DGBRZ => 6 ZKZNB\n7 GDBNV, 1 QCGM => 8 QFJS\n2 QFQPN, 5 JWLZ => 4 NSVG\n8 QFJS, 1 ZDZM, 4 QSLTQ => 7 MVSHM\n1 LTVKM => 8 RFZF\n4 DNWGS => 3 FTNZQ\n6 VZJHN => 9 PTDL"

In [5]:
parsed_input = parse_input(input)

65-element Vector{SubString{String}}:
 "2 WZMS, 3 NPNFD => 5 SLRGD"
 "4 QTFCJ, 1 RFZF => 1 QFQPN"
 "2 LCDPV => 6 DGPND"
 "1 MVSHM, 3 XSDR, 1 RSJD => 6 GNKB"
 "6 XJRML, 1 LCDPV => 7 HTSJ"
 "3 LQBX => 3 GKNTG"
 "2 NZMLP, 5 FTNZQ => 2 QSLTQ"
 "8 WZMS, 4 XSDR, 2 NPNFD => 9 CJVT"
 "16 HFHB, 1 TRVQG => 8 QTBQ"
 "177 ORE => 7 DNWGS"
 "10 ZJFM, 4 MVSHM => 8 LCDPV"
 "1 LTVKM => 5 ZJFM"
 "5 QFJS => 6 LTVKM"
 ⋮
 "9 NZMLP, 5 FQCKM, 8 QFJS => 1 LQBX"
 "170 ORE => 9 GDBNV"
 "5 RSJD, 3 CZHM, 1 GNKB => 6 HNJW"
 "14 HTSJ => 7 FXJQX"
 "11 NPNFD, 1 LCDPV, 2 FXJQX => 6 RSJD"
 "9 DGBRZ => 6 ZKZNB"
 "7 GDBNV, 1 QCGM => 8 QFJS"
 "2 QFQPN, 5 JWLZ => 4 NSVG"
 "8 QFJS, 1 ZDZM, 4 QSLTQ => 7 MVSHM"
 "1 LTVKM => 8 RFZF"
 "4 DNWGS => 3 FTNZQ"
 "6 VZJHN => 9 PTDL"

In [6]:
sample = """10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL"""

"10 ORE => 10 A\n1 ORE => 1 B\n7 A, 1 B => 1 C\n7 A, 1 C => 1 D\n7 A, 1 D => 1 E\n7 A, 1 E => 1 FUEL"

In [7]:
parsed_sample = parse_input(sample)

6-element Vector{SubString{String}}:
 "10 ORE => 10 A"
 "1 ORE => 1 B"
 "7 A, 1 B => 1 C"
 "7 A, 1 C => 1 D"
 "7 A, 1 D => 1 E"
 "7 A, 1 E => 1 FUEL"

In [8]:
function parse_numbered_item(s)
    n, item = split(s)
    return (item, parse(Int, n))
end

parse_numbered_item (generic function with 1 method)

In [9]:
function preprocess(input)
      dict = Dict{String, Vector{Tuple{String, Int}}}()
    adj = DefaultDict{String, Vector{String}}(() -> String[])

    foreach(input) do line
        left, right = split(line, " => ")
        ri, rn = parse_numbered_item(right)
        dict[ri] = parse_numbered_item.(split(left, ", "))
        for (li, _) in dict[ri]
            push!(adj[li], ri)
        end
        push!(dict[ri], (ri, rn))
    end

    @assert length(dict) == length(input)

    known = DefaultDict{String, Int}(() -> 0)
    q = Queue{String}()
    enqueue!(q, "ORE")
    order = ["ORE"]
    while !isempty(q)
        u = dequeue!(q)
        for v in adj[u]
            known[v] += 1
            if known[v] == length(dict[v]) - 1
                push!(order, v)
                enqueue!(q, v)
            end
        end
    end

    return dict, order
end

preprocess (generic function with 1 method)

In [10]:
function solve(dict, order, num)
    req = DefaultDict{String,Int}(() -> 0)
    req["FUEL"] = num

    for item in order[end:-1:1]
        if haskey(req, item) && item != "ORE"
            n = req[item]
            per = dict[item][end][2]
            multiply = (n - 1) ÷ per + 1
            for (li, ln) in dict[item][1:end-1]
                req[li] += multiply * ln
            end
        end
    end

    return req["ORE"]
end

solve (generic function with 1 method)

In [11]:
function part_one(input)
    dict, order = preprocess(input)
    return solve(dict, order, 1)
end

part_one (generic function with 1 method)

In [12]:
part_one(parsed_sample)

31

In [13]:
part_one_ans = part_one(parsed_input)

201324

In [14]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 14 - Advent of Code 2019</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3307 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [15]:
function part_two(input)
    dict, order = preprocess(input)
    lo = 1
    hi = Int(1e12)
    while lo <= hi
        mid = (lo + hi) >> 1
        req = solve(dict, order, mid)
        if req <= Int(1e12)
            lo = mid + 1
        else
            hi = mid - 1
        end
    end
    return hi
end

part_two (generic function with 1 method)

In [16]:
part_two(parsed_sample)

34482758620

In [17]:
part_two_ans = part_two(parsed_input)

6326857

In [18]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 14 - Advent of Code 2019</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3258 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"